LLM 에이전트 : LLM이 외부 도구를 활용해서 복잡한 테스크를 수행하는 시스템

- 작동흐름: 사용자 요청 -> LLM(추론) -> 도구선택 -> 도구 실행 -> 결과 해석 -> 응답

- 에이전트 대화 패턴
user <-> Assistant
사용자가 요청 -> 어시스턴스 응답 -> 사용자 피드백 반영

- 각 에이전트가 전문 영역을 담당
Manager -> Agent1 -> Agent2 -> Agent3 -> Manager
→ Manager가 전체 작업 조율, Agent는 세부 작업 수행

AutoGen 프레임워크 : LLM을 기반으로 한 다중 에이전트 시스템을 쉽게 만들 수 있게 해주는 오픈소스 프레임워크

- 핵심 기능
    - 에이전트 간 협업
    - Agent들이 대화를 통해 역할 분담 및 협업 수행
    - LLM 지시 기반으로 코드 작성 + 실행 가능
- 에이전트 종류
    - AssistantAgent 지시에 따라서 작업수행
    - UserProxyAgent 사용자 역할, 코드 실행
    - GroupChatManager : 여러 에이전트 조율

In [ ]:
%pip install autogen

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# api 설정
config_list = [
    {
        'model' : 'gpt-4o-mini',
        'api_key': os.environ['OPENAI_API_KEY']
    }
]
llm_config = {
    'config_list' : config_list,
    'temperature' : 0
}
# 어시스턴스 에이전트 생성
import autogen
assistant = autogen.AssistantAgent(
    name = 'Assistant',
    llm_config = llm_config,
    system_message = 'You are a helpful assistant.'
)
# user proxy 에이전트
user_proxy = autogen.UserProxyAgent(
  name = 'User_proxy',
  human_input_mode = 'ALWAYS', # 자동실행
  code_execution_config={
      'work_dir' : 'coding',
      'use_docker' : False
  })
# 대화
user_proxy.initiate_chat(
    assistant,
    message = '파이썬으로 팩토리얼 함수 작성해줘',
    single_round = True) # 단일 라운드만 수행

In [ ]:
# 코드 실행 에이전트
import autogen
# 종료조건 설정
def is_termination_msg(msg):
  content = msg.get('content',"")
  return content and content.strip().endswith('TERMINATE')

# 어시스턴스 에이전트 생성
import autogen
assistant = autogen.AssistantAgent(
    name = 'Assistant',
    llm_config = llm_config,
    system_message = 'You are a helpful assistant.'
)
# user proxy 에이전트
user_proxy = autogen.UserProxyAgent(
  name = 'User_proxy',
  is_termination_msg = is_termination_msg,
  human_input_mode = 'NEVER',

  code_execution_config={
      'work_dir' : 'workspace',
      'use_docker' : False
  })

# 대화
user_proxy.initiate_chat(
    assistant,
    message = '삼성전쟈의 최근 3개월치 주식 데이터를 가져와서 분석하고 시각화 해줘',
)